# Sphere versus Torus

In this notebook, we attempt to use persistence landscapes to determine if TDA can tell the difference between a sphere and a torus. We sample 1000 points from each shape, compute the Vietoris-Rips persistent homology using Ripser, and then compute the respective landscapes. 

To determine if we can see a difference, we will repeat this process 100 times and then compute the mean landscape across these 100 runs.

...Finish intro...

#### Imports 

In [1]:
import numpy as np
from ripser import ripser
from tadasets import sphere, torus
from PersistenceLandscapeGrid import PersistenceLandscapeGrid
from operator import itemgetter

In [2]:
sphere_pts = sphere(n=1000)
torus_pts = torus(n=1000)

In [3]:
sphere_dgms = ripser(sphere_pts)['dgms']
torus_dgms = ripser(torus_pts)['dgms']

At this point, `PersistenceLandscapeGrid` requires the user to specify the parameters of a grid explicitly. Since we want to compare the diagrams from the sphere and torus, we compute the appropriate start and stop point.

In [4]:
hom_deg = 1
sph_b = min(sphere_dgms[hom_deg],key=itemgetter(0))[0]
sph_d = max(sphere_dgms[hom_deg],key=itemgetter(1))[1]
tor_b = min(torus_dgms[hom_deg],key=itemgetter(0))[0]
tor_d = max(torus_dgms[hom_deg],key=itemgetter(1))[1]

start = min(sph_b,tor_b)
stop = max(sph_d,tor_d)

In [5]:
sph_pl = PersistenceLandscapeGrid(start=start,
                                 stop=stop,
                                 num_dims=500,
                                 diagrams=sphere_dgms,
                                 homological_degree=1)
tor_pl = PersistenceLandscapeGrid(start=start,
                                 stop=stop,
                                 num_dims=500,
                                 diagrams=torus_dgms,
                                 homological_degree=1)

In [6]:
sph_pl.compute_landscape()
tor_pl.compute_landscape()

In [7]:
diff_pl = sph_pl - tor_pl

(Put picture of diff here maybe, once the code is done.)

In [12]:
significant = diff_pl.sup_norm()

print(f'The threshold for significance is {significant}')

The threshold for significance is 0.7291105946895474


## 100 Runs

This is a single run. Now lets do it 100 times. This might be very slow. One way to speed it up is multiprocessing but we skip that for now.

Let's compute the start and stop parameters of our grid on the fly to save time.

In [15]:
sph_dgm_list = []
tor_dgm_list = []
sph_b = 100.
sph_d = -100.
tor_b = 100.
tor_d = -100.

for i in range(100):
    sph_pts = sphere(n=1000)
    sph_dgm = ripser(sph_pts)['dgms']
    if min(sph_dgm[1],key=itemgetter(0))[0] < sph_b:
        sph_b = min(sph_dgm[1],key=itemgetter(0))[0]
    if max(sph_dgm[1],key=itemgetter(1))[1] > sph_d:
        sph_d = max(sph_dgm[1],key=itemgetter(1))[1]
    sph_dgm_list.append(sph_dgm)
    
    tor_pts = torus(n=1000)
    tor_dgm = ripser(tor_pts)['dgms']
    if min(tor_dgm[1],key=itemgetter(0))[0] < tor_b:
        tor_b = min(tor_dgm[1],key=itemgetter(0))[0]
    if max(tor_dgm[1],key=itemgetter(1))[1] > tor_d:
        tor_d = max(tor_dgm[1],key=itemgetter(1))[1]
    tor_dgm_list.append(tor_dgm)

KeyboardInterrupt: 